# Searching For Parking Dataset 

This is from Geotab's and Smart Columbus's Searching For Parking dataset. More information about the calculation is available here: (https://data.geotab.com/urban-infrastructure/searching-for-parking) and the initial dataset is shown below. I (Derek Topper) have parsed that dataset down to the needed variables for this project to generate a simple score that a user can use to locate a zone, or geohash, where there is likely to be more parking.


In [54]:
import pandas as pd
import numpy as np
import geopy.distance

In [55]:
#Full Dataset
pd.read_csv('95146f08-cda0-4ae6-87f9-1e5fc49b1709.csv').tail(3)

,parkinggeohash,avglatitude,avglongitude,city,county,state,country,iso_3166_2,avgtimetopark,avgtimetoparkratio,...,circlingdistribution,hourlydistribution,percentcar,percentmpv,percentldt,percentmdt,percenthdt,percentother,updatedate,version
825,dphgvez,40,-83,Columbus,Franklin,Ohio,United States of America,US-OH,5,1,...,"{""dphgvsp"":0.1519, ""dphgvey"":0.1223, ""dphgvez""...","{""00"": null, ""01"": null, ""02"": null, ""03"": nul...",0,0,0,0,0,0,2018-05-02,1
826,dphgkn6,39,-83,Columbus,Franklin,Ohio,United States of America,US-OH,5,1,...,"{""dphgkn7"":0.087, ""dphgknd"":0.07, ""dphgkn6"":0....","{""00"": null, ""01"": 0.125, ""02"": null, ""03"": nu...",0,0,0,0,0,0,2018-05-02,1
827,dphgr3d,39,-82,Columbus,Franklin,Ohio,United States of America,US-OH,5,1,...,"{""dphgr3c"":0.1088, ""dphgr3e"":0.1442, ""dphgr3g""...","{""00"": null, ""01"": null, ""02"": null, ""03"": nul...",0,0,0,0,0,0,2018-05-02,1


In [56]:
#Small Dataset of Just The Stuff We Care About Currently

#This has no duplicates. The one above does. 

df = pd.read_csv("SmallSearchingForParking.csv").set_index("i")
df.head(5)

,ParkingGeohash,AvgLatitude,AvgLongitude,AvgTimeToPark,AvgTotalGeohashes,TotalSearching
i,,,,,,
0,dpj55nh,39.93588,-82.83093,5.8639,7.1667,6
2,dphg7yw,39.98270,-83.14658,7.0306,9.5000,6
3,dphgr5u,39.96765,-83.00645,6.0729,11.0000,8
4,dphgsdg,40.00588,-83.11744,4.4111,9.3333,6
5,dphgvtg,40.06605,-83.07390,3.4738,6.3571,14


In [57]:
#DISTANCE TO FOR EACH GEOHASH TO A SPECIFIC COORDINATE
def distance_from_geohashes(x_coord, y_coord):
    coords_2 = (x_coord, y_coord)
    arr = []
    for each in np.arange(len(df)):
        coords_1 = (df.iloc[each][1], df.iloc[each][2])
        arr.append(geopy.distance.vincenty(coords_1, coords_2).km)
    return(arr)

print("MIN:" , min(distance_from_geohashes(40, -83)))
print("MAX:" , max(distance_from_geohashes(40, -83)))

MIN: 0.7790782667370509
MAX: 18.28403542070697


C:\Users\Derek\Miniconda3\envs\ds100\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  import sys


In [58]:
#This is a quick score to find the best geohash to look for parking
#Lower score is better

#Score is basically Average Time to Park PLUS
#  the Distance to the Coord We Want times 2.5 mins (the average time to travel a KM in the city)

d=distance_from_geohashes(40, -83)
dist = (2.5*np.asarray(d))
avgpark = df["AvgTimeToPark"]

best_geohashes_toseek_parking = dist + avgpark
np.sort(best_geohashes_toseek_parking)[0:5]

C:\Users\Derek\Miniconda3\envs\ds100\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  import sys


array([5.98769567, 8.15162074, 8.22769805, 9.46427169, 9.49701609])

In [59]:
#This is the score abstraction. It takes the coords and outputs the 5 nearest geohashes (as df).

def score_abstraction(x_coord, y_coord):
    dis = np.asarray(distance_from_geohashes(x_coord, y_coord))
    df["DisToCoord"] = dis       
    distanco = 2.5*dis
    avgpark = df["AvgTimeToPark"]
    best_geohashes_toseek_parking = dist + avgpark
    df["Score"] = best_geohashes_toseek_parking
    return df.sort_values("Score").head(5)
score_abstraction(40, -83)

C:\Users\Derek\Miniconda3\envs\ds100\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  import sys


,ParkingGeohash,AvgLatitude,AvgLongitude,AvgTimeToPark,AvgTotalGeohashes,TotalSearching,DisToCoord,Score
i,,,,,,,,
32,dphgx47,40.00293,-83.00829,4.0400,7.8000,5,0.779078,5.987696
26,dphgwbz,39.99485,-83.01370,4.8961,7.4118,17,1.302208,8.151621
134,dphgrpv,39.98929,-83.00533,5.0444,8.2222,9,1.273319,8.227698
39,dphgx49,40.00435,-83.01017,6.9800,9.0000,5,0.993709,9.464272
161,dphgwbt,39.99399,-83.01573,5.7472,10.1667,6,1.499926,9.497016


In [67]:
#This is a score abstraction. It takes the coords and outputs the 
#                          5 nearest geohash coords (as list).

def score_abstraction(x_coord, y_coord):
    dis = np.asarray(distance_from_geohashes(x_coord, y_coord))
    df["DisToCoord"] = dis       
    distanco = 2.5*dis
    avgpark = df["AvgTimeToPark"]
    best_geohashes_toseek_parking = dist + avgpark
    df["Score"] = best_geohashes_toseek_parking

    five = []
    for each in np.arange(5):
        five.append([df.sort_values("Score").iloc[each][1],
                     df.sort_values("Score").iloc[each][2],
                     df.sort_values("Score").iloc[each][7]])
    return five
score_abstraction(40, -83)

C:\Users\Derek\Miniconda3\envs\ds100\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  import sys


[[40.00293, -83.00829, 5.987695666842628],
 [39.99485, -83.0137, 8.151620743002736],
 [39.98929, -83.00533, 8.227698047008001],
 [40.00435, -83.01017, 9.464271689043812],
 [39.99399, -83.01573, 9.497016085326848]]